<a href="https://colab.research.google.com/github/SiyuanChen726/BreastAgeNet/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SiyuanChen726/BreastAgeNet.git

Cloning into 'BreastAgeNet'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 295 (delta 65), reused 8 (delta 7), pack-reused 165 (from 1)
Receiving objects: 100% (295/295), 24.67 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [ ]:
!apt-get update
!apt-get install -y openslide-tools libopenslide0
!pip install openslide-python

In [ ]:
# Download the files with corrected Google Drive links
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1tDfFXZvB6PASPW-4OMrBU-HQo7optH53' -O 'BreastAgeNet_bestModel.pt'

--2024-11-30 06:15:45--  https://drive.google.com/uc?export=download&id=1tDfFXZvB6PASPW-4OMrBU-HQo7optH53
Resolving drive.google.com (drive.google.com)... 74.125.68.139, 74.125.68.138, 74.125.68.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.68.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1tDfFXZvB6PASPW-4OMrBU-HQo7optH53&export=download [following]
--2024-11-30 06:15:45--  https://drive.usercontent.google.com/download?id=1tDfFXZvB6PASPW-4OMrBU-HQo7optH53&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.175.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.175.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9995011 (9.5M) [application/octet-stream]
Saving to: ‘BreastAgeNet_bestModel.pt’

BreastAgeNet_bestMo 100%[===================>]   9.53M 

In [ ]:
import os
import sys
sys.path.append('/content/BreastAgeNet')

import openslide
import h5py
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


from utils_vis import *
from utils_train import *

In [ ]:
ckpt_name = "BreastAgeNet_bestModel.pt"
n_feats = get_dim_input(model_name = "UNI")

model = BreastAgeNet(n_feats, attention="MultiHeadAttention", n_classes=3, n_heads=8, n_latent=512, embed_attn=True)
model.load_state_dict(torch.load(ckpt_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

<ipython-input-2-aa43e33e99df>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_name))


cuda


In [ ]:
clinic_df = pd.read_csv("/scratch/prj/cb_normalbreast/Siyuan/prj_normal/BreastAgeNet/DATA/clinicData/AllCohorts.csv")
clinic_df = clinic_df.loc[clinic_df["cohort"]=="development", :].copy()

valid_patches = []
for fea_pt in clinic_df.h5df.values:
    with h5py.File(fea_pt, "r") as file:
        bag = np.array(file["embeddings"])
        bag = np.squeeze(bag)
        img_id = np.array(file["patch_id"])
    img_id = [i.decode("utf-8") for i in img_id]
    bag_df = pd.DataFrame(bag)
    bag_df.index = img_id

    csv_pt =  os.path.join(Path(fea_pt).parent, Path(fea_pt).stem.split('_bagFeature_')[0]+'_patch.csv')
    df = pd.read_csv(csv_pt)
    valid_id = list(df['patch_id'][df['TC_epi'] > 0.9])
    valid_id = list(set(valid_id) & set(bag_df.index))
    valid_patches.extend(valid_id)
print(len(valid_patches))

valid_wsis = ["_".join(i.split("_")[:3]) for i in valid_patches]
print(len(np.unique(valid_wsis)))

a, b = np.unique(valid_wsis, return_counts=True)
filtered_a = [i for i, count in zip(a, b) if count >= 5]
print(len(filtered_a))
print(len(np.unique([i.split("_")[0] for i in filtered_a])))
clinic_df = clinic_df[clinic_df["wsi_id"].isin(filtered_a)]
print("Filtered DataFrame length:", len(clinic_df))
print("Unique age groups and counts:", np.unique(clinic_df["age_group"], return_counts=True))


print(len(valid_patches))
valid_patches = [i for i in valid_patches if "_".join(i.split("_")[:3]) in filtered_a]
print(len(valid_patches))

In [ ]:
df = clinic_df.copy() # file containing all {wsi_id}_UNI.h5 files for the dataset
df['h5df'] = [Path(i) for i in df["h5df"]]

valid_patches = valid_patches # select patches with TC_epi > 0.9
test_dblock = DataBlock(blocks = (TransformBlock, CategoryBlock),
               get_x = ColReader('h5df'),
               get_y = ColReader('age_group'),
               item_tfms = MILBagTransform(df.h5df, bag_size, valid_patches))

test_dls = test_dblock.dataloaders(df, bs=bag_size, shuffle=False)
dataloaders = test_dls.test_dl(df, with_labels=True)

In [ ]:
# Initialize test loader, assume clinic_df is pre-defined
phase = 'test'
model.eval()

predicted_ranks = []
truelabels = []
df = pd.DataFrame()  # Initialize empty DataFrame

# Loop through the test data
for repeat in range(10):
    for (patch_ids, inputs), labels in tqdm(dataloaders):  # Ensure 'dataloaders' is correctly defined elsewhere
        patch_ids = np.array(patch_ids)  # Shape: (150, 7)
        patch_ids = np.transpose(patch_ids)  # Shape: (7, 150)
        patch_ids = patch_ids.flatten()  # Flattened to shape (1050,)

        # Enable gradients only if in training phase
        with torch.set_grad_enabled(phase == 'train'):
            logits, embeddings, attentions = model(inputs)

            attentions = attentions.view(-1, attentions.shape[-1])  # Flatten attentions
            embeddings = embeddings.view(-1, embeddings.shape[-1])  # Flatten embeddings
            probs = torch.sigmoid(logits)  # Shape: [batch_size, n_classes]
            binary_predictions = (probs > 0.5).int()  # Shape: [batch_size, n_classes]
            ranks = binary_predictions.sum(dim=1)  # Sum the predictions to get a rank score
            predicted_ranks += ranks.tolist()  # Accumulate predicted ranks
            truelabels += labels.tolist()  # Accumulate true labels

        # Concatenate patch_ids, embeddings, and attentions
        # Assuming patch_ids, embeddings, and attentions are of compatible shapes
        combined_data = np.column_stack((repeat, patch_ids, embeddings.cpu().numpy(), attentions.cpu().numpy()))  # Convert to numpy
        dfi = pd.DataFrame(combined_data, columns=['repeat'] + ['patch_id'] +
                                                    [f'embedding_{i}' for i in range(embeddings.shape[1])] +
                                                    [f'attention_{i}' for i in range(attentions.shape[1])])

        # Append new data to the main DataFrame
        df = pd.concat([df, dfi], axis=0, ignore_index=True)

# Remove duplicates from the final DataFrame
df = df.drop_duplicates()

In [ ]:
MAE = np.abs(truelabels - np.array(predicted_ranks)).mean()
print(f'Mean Absolute Error (MAE): {MAE}')

In [ ]:
# average for each WSI

In [ ]:
# vote for each patient

from scipy.special import expit  # For sigmoid function

# Step 1: Convert 'branch_0', 'branch_1', 'branch_2' to numeric (if necessary)
outputs_WSI['branch_0'] = pd.to_numeric(outputs_WSI['branch_0'], errors='coerce')
outputs_WSI['branch_1'] = pd.to_numeric(outputs_WSI['branch_1'], errors='coerce')
outputs_WSI['branch_2'] = pd.to_numeric(outputs_WSI['branch_2'], errors='coerce')

# Step 2: Group by 'wsi_id' and calculate the mean for each branch across repeats
averaged_data = outputs_WSI.groupby('wsi_id')[['branch_0', 'branch_1', 'branch_2']].mean().reset_index()

# Step 3: Apply sigmoid to the averaged values
averaged_data['sigmoid_0'] = expit(averaged_data['branch_0'])  # Sigmoid for branch 0
averaged_data['sigmoid_1'] = expit(averaged_data['branch_1'])  # Sigmoid for branch 1
averaged_data['sigmoid_2'] = expit(averaged_data['branch_2'])  # Sigmoid for branch 2

# Step 4: Convert sigmoid values to binary (0 or 1)
averaged_data['binary_0'] = (averaged_data['sigmoid_0'] >= 0.5).astype(int)
averaged_data['binary_1'] = (averaged_data['sigmoid_1'] >= 0.5).astype(int)
averaged_data['binary_2'] = (averaged_data['sigmoid_2'] >= 0.5).astype(int)

# Step 5: Sum the binary predictions to get the final prediction
averaged_data['final_prediction'] = averaged_data[['binary_0', 'binary_1', 'binary_2']].sum(axis=1)

# Display the results (only final prediction for simplicity)
print(averaged_data[['wsi_id', 'final_prediction']])



In [ ]:
df["wsi_id"] = [i.split("_")[0] for i in list(df["patch_id"])]
df = pd.merge(df, clinic_df, on="wsi_id")